In [65]:
from config import db_config
from collections import defaultdict
import pyodbc
from beem.account import Account

In [66]:
connection = pyodbc.connect(driver=db_config['driver'],
                            server=db_config['server'],
                            database=db_config['database'],
                            uid=db_config['uid'],
                            pwd=db_config['pwd'])
cursor = connection.cursor()

In [67]:
def to_sql_list(x):
    return "('" + "', '".join(x) + "')"

In [68]:
query = """\
SELECT TOP 100 account
FROM (
       SELECT SUBSTRING(body, CHARINDEX('@', body) + 1, CHARINDEX(' leads', body) - CHARINDEX('@', body) - 1) as account, *
       FROM Comments (NOLOCK)
       WHERE depth > 0 AND
             author = 'guard' AND CONTAINS(body, 'phishing')) C
GROUP BY account
ORDER BY COUNT(*) DESC"""

scammers = [row[0] for row in cursor.execute(query)]
with open('scammers.txt', 'w') as f:
    f.write('\n'.join(scammers))

In [69]:
query = """\
SELECT TOP 100 author
FROM Comments (NOLOCK)
WHERE depth = 0 AND
      category in ('utopian-io', 'dtube', 'dlive')
ORDER BY NEWID()"""

content_creators = [row[0] for row in cursor.execute(query)]
with open('content_creators.txt', 'w') as f:
    f.write('\n'.join(content_creators))

In [70]:
query = """\
SELECT TOP 100 author
FROM Comments (NOLOCK)
WHERE depth = 1 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE() AND
      ((CONTAINS (body, 'nice') AND body LIKE 'nice') OR
       (CONTAINS (body, 'beautiful') AND body LIKE 'beautiful') OR
       (CONTAINS (body, 'upvoted') AND body LIKE 'upvoted'))
GROUP BY author
ORDER BY COUNT(*) DESC"""

comment_spammers = [row[0] for row in cursor.execute(query)]
with open('comment_spammers.txt', 'w') as f:
    f.write('\n'.join(comment_spammers))

In [71]:
bid_bots = list(map(str.strip, open('bid_bots.txt', 'r').readlines()))
full_list = scammers + content_creators + comment_spammers + bid_bots
full_list = full_list[:20]
d = defaultdict(lambda: defaultdict(int))

In [72]:
for name in full_list:
    print(name)
    account = Account(name)
    foll = account.get_follow_count()
    d[name]['followers'] = foll['follower_count']
    d[name]['followings'] = foll['following_count']
    d[name]['follow ratio'] = foll['following_count'] / foll['follower_count']
    d[name]['muters'] = len(account.get_muters())
    d[name]['reputation'] = account.get_reputation()
    d[name]['effective sp'] = account.get_steem_power()
    own_sp = account.get_steem_power(onlyOwnSP=True)
    d[name]['own sp'] = own_sp
    d[name]['sp ratio'] = account.get_steem_power() / own_sp if own_sp > 0 else 0

erudire


shovon51
ilhamyus
nirayma
masjuan
marlon23
blockchainfiend
rahmad07
sanjugangber
upvotebott
attakhan
miky
udywriter
yanisteem
jennyluv
mujib
ayumanis
asimmons8000
zulfajri321
muksalpasee


In [73]:
query = """\
SELECT
  name,
  curation_rewards,
  posting_rewards,
  witnesses_voted_for
FROM Accounts (NOLOCK) a
WHERE name in """ + to_sql_list(full_list)

for row in cursor.execute(query):
    name = row[0]
    curation_rewards = row[1] / 1000.0
    posting_rewards = row[2] / 1000.0
    witnesses_voted_for = row[3]   
    d[name]['curation_rewards'] = curation_rewards
    d[name]['posting_rewards'] = posting_rewards
    d[name]['witnesses_voted_for'] = witnesses_voted_for

In [74]:
query = """\
SELECT
  author,
  COUNT(*),
  AVG(LEN(body))
FROM Comments (NOLOCK) a
WHERE depth = 0 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE()
      AND author in """ + to_sql_list(full_list) + """
GROUP BY author"""

In [75]:
for row in cursor.execute(query):
    name = row[0]
    posts = row[1]
    average_post_len = row[2]
    d[name]['posts'] = posts
    d[name]['average_post_len'] = average_post_len

In [76]:
query="""\
SELECT
  author,
  COUNT(*),
  AVG(LEN(body)),
  CAST(SUM(CASE WHEN body LIKE '%http%' THEN 1 ELSE 0 END) as DECIMAL(10, 3)) / COUNT(*)
FROM Comments (NOLOCK) a
WHERE depth > 0 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE()
      AND author in """ + to_sql_list(full_list) + """
GROUP BY author
"""

for row in cursor.execute(query):
    name = row[0]
    comments = row[1]
    average_comment_len = row[2]
    comments_with_link_ratio = row[3]
    d[name]['comments'] = comments
    d[name]['average_comment_len'] = average_comment_len
    d[name]['comments_with_link_ratio'] = comments_with_link_ratio
    d[name]['posts_to_comments_ratio'] = d[name]['posts'] / comments if comments > 0 else 0

In [77]:
for k, v in d.items():
    print(k)
    for k2, v2 in v.items():
        print('\t', k2, ':', v2)

erudire
	 followers : 329
	 followings : 171
	 follow ratio : 0.5197568389057751
	 muters : 24
	 reputation : -7.788755198572268
	 effective sp : 15.012604334181919
	 own sp : 0.585694049908604
	 sp ratio : 25.632161256418087
	 curation_rewards : 0.022
	 posting_rewards : 0.117
	 witnesses_voted_for : 3
	 posts : 23
	 average_post_len : 6562
	 comments : 1571
	 average_comment_len : 188
	 comments_with_link_ratio : 0.99172501591343
	 posts_to_comments_ratio : 0.01464035646085296
shovon51
	 followers : 204
	 followings : 15
	 follow ratio : 0.07352941176470588
	 muters : 18
	 reputation : -3.3902649952616635
	 effective sp : 15.053853339149454
	 own sp : 2.3852650310162185
	 sp ratio : 6.311186867455106
	 curation_rewards : 0.004
	 posting_rewards : 3.709
	 witnesses_voted_for : 3
	 posts : 48
	 average_post_len : 300
	 comments : 935
	 average_comment_len : 79
	 comments_with_link_ratio : 0.88021390374331
	 posts_to_comments_ratio : 0.051336898395721926
ilhamyus
	 followers : 136
	 fol